In [1]:
#envs/py36tf

import re

import pandas as pd
import numpy as np

In [35]:
fn = 'Map Revisions.xlsx'

df = pd.read_excel(fn)

In [36]:
df

,Market,MSO,Comcast Division,AT&T,Verizon,T-Mobile,Sprint,Other,Unnamed: 8,Unnamed: 9
0,"Anderson, IN",Comcast,Central,5G Evolution April 2018,NaN,NaN,NaN,NaN,NaN,NaN
1,"Ann Arbor, MI",Comcast,Central,NaN,Pre 5G fixed trial May 2017,NaN,NaN,NaN,NaN,NaN
2,"Atlanta, GA",Comcast/Charter,Central,"Launch 5G Mobile Dec. 21, 2018;\n5G Evolution ...",Pre 5G fixed trial Aug. 2017,NaN,Launch Massive MIMO – YE 2018;\nLaunch 5G Mobi...,NaN,NaN,NaN
3,"Aurora, IL",Comcast,Central,5G Evolution April 2018,NaN,NaN,NaN,NaN,NaN,NaN
4,"Benton Harbor, MI",Comcast,Central,5G Evolution October 2018,NaN,NaN,NaN,NaN,NaN,NaN
5,"Bloomington, IN",Comcast,Central,5G Evolution April 2018,NaN,NaN,NaN,NaN,NaN,NaN
6,"Bureau, IL",Comcast,Central,5G Evolution October 2018,NaN,NaN,NaN,NaN,NaN,NaN
7,"Charleston, SC",Comcast/Charter,Central,5G Evolution October 2018,NaN,NaN,NaN,NaN,NaN,NaN
8,"Chicago, IL",Comcast,Central,5G Evolution YE 2017,NaN,NaN,Launch Massive MIMO – April 2018,NaN,NaN,NaN
9,"Citrus, FL",Comcast/Charter,Central,5G Evolution October 2018,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
to_keep = [x for x in df.columns if 'Unnamed' not in x]
df = df[to_keep].copy()

In [38]:
df.head()

,Market,MSO,Comcast Division,AT&T,Verizon,T-Mobile,Sprint,Other
0,"Anderson, IN",Comcast,Central,5G Evolution April 2018,NaN,NaN,NaN,NaN
1,"Ann Arbor, MI",Comcast,Central,NaN,Pre 5G fixed trial May 2017,NaN,NaN,NaN
2,"Atlanta, GA",Comcast/Charter,Central,"Launch 5G Mobile Dec. 21, 2018;\n5G Evolution ...",Pre 5G fixed trial Aug. 2017,NaN,Launch Massive MIMO – YE 2018;\nLaunch 5G Mobi...,NaN
3,"Aurora, IL",Comcast,Central,5G Evolution April 2018,NaN,NaN,NaN,NaN
4,"Benton Harbor, MI",Comcast,Central,5G Evolution October 2018,NaN,NaN,NaN,NaN


In [39]:
df = pd.melt(df, id_vars=['Market', 'MSO'], value_vars = ['AT&T', 'Verizon', 'T-Mobile', 'Sprint'], var_name='5G Provider', value_name='5G status')

In [40]:
df.head()

,Market,MSO,5G Provider,5G status
0,"Anderson, IN",Comcast,AT&T,5G Evolution April 2018
1,"Ann Arbor, MI",Comcast,AT&T,NaN
2,"Atlanta, GA",Comcast/Charter,AT&T,"Launch 5G Mobile Dec. 21, 2018;\n5G Evolution ..."
3,"Aurora, IL",Comcast,AT&T,5G Evolution April 2018
4,"Benton Harbor, MI",Comcast,AT&T,5G Evolution October 2018


In [41]:

df = df[~(pd.isnull(df['5G status'])) & (df['5G status'].str.strip() != '')].copy()

In [42]:
df.head()

,Market,MSO,5G Provider,5G status
0,"Anderson, IN",Comcast,AT&T,5G Evolution April 2018
2,"Atlanta, GA",Comcast/Charter,AT&T,"Launch 5G Mobile Dec. 21, 2018;\n5G Evolution ..."
3,"Aurora, IL",Comcast,AT&T,5G Evolution April 2018
4,"Benton Harbor, MI",Comcast,AT&T,5G Evolution October 2018
5,"Bloomington, IN",Comcast,AT&T,5G Evolution April 2018


In [43]:
df[df.Market.str.contains('Sacramento')]

,Market,MSO,5G Provider,5G status
70,"Sacramento, CA",Comcast,AT&T,5G Evolution YE 2017
305,"Sacramento, CA",Comcast,Verizon,Launch 5G Home fixed wireless service Oct. 201...


In [44]:
re.sub(';\n', '; ', 'Massive begun April 2018;\n5G Mobile - 1H 2019')

'Massive begun April 2018; 5G Mobile - 1H 2019'

In [45]:
def fix_status(obj):
    txt = re.sub('  ', '', obj)
    txt = re.sub('[; \n]{2,}', '; ', txt)
    txt = re.sub('\n', '; ', txt)
    return txt.strip()

df['5G status'] = df['5G status'].apply(fix_status)

In [46]:
df['5G Provider'].unique()

array(['AT&T', 'Verizon', 'T-Mobile', 'Sprint'], dtype=object)

In [47]:
img_d = {
    'AT&T': 'https://static1.squarespace.com/static/58f5270c17bffcda49872c1a/5b490bd770a6adf3c3a50646/5b900deec2241b406da03ff1/1536167410013/AT%26T.png?format=750w',
    'Sprint': 'https://static1.squarespace.com/static/58f5270c17bffcda49872c1a/5b490bd770a6adf3c3a50646/5b900deecd836633a4544d78/1536167412853/Sprint.png?format=750w',
    'T-Mobile': 'https://static1.squarespace.com/static/58f5270c17bffcda49872c1a/5b490bd770a6adf3c3a50646/5b900df088251bb719f50d44/1536167412451/T-Mobile.png?format=750w',
    'Verizon': 'https://static1.squarespace.com/static/58f5270c17bffcda49872c1a/5b490bd770a6adf3c3a50646/5b900df0758d4602635ba1d8/1536167411876/Verizon+5G.png?format=750w'
}

def add_imgs(obj):
    return img_d[obj]

df['img_url'] = df['5G Provider'].apply(add_imgs)

In [48]:
df[df.Market.str.contains('Alpine')]

,Market,MSO,5G Provider,5G status,img_url
86,"Alpine, CA",Other,AT&T,5G Evolution April 2018,https://static1.squarespace.com/static/58f5270...


In [49]:
df['MSO'].unique()

array(['Comcast', 'Comcast/Charter', 'Comcast/Cable One', 'Comcast/Cox',
       'Altice', 'Other', 'Charter', 'GCI', 'Cable One', 'Cox',
       'Charter/Cox', 'Charter/Altice'], dtype=object)

In [50]:
['Comcast', 'Charter', 'Altice', 'No MSO', 'GCI', 'Cable One', 'Cox']

['Comcast', 'Charter', 'Altice', 'No MSO', 'GCI', 'Cable One', 'Cox']

In [51]:
df['5G status'] = df['5G status'].str.replace('5G fixed', '5G Fixed')


In [52]:
df['5G status'] = df['5G status'].str.replace('5G Evolution', 'Mobile 5G Evolution')

In [53]:
def get_launch_trial(obj):
    if re.match('launch', obj.lower()):
        return 'Launch'
    else:
        return 'Pre 5G / Trial'

df['launch_type'] = df['5G status'].apply(get_launch_trial)

In [54]:
df.columns

Index(['Market', 'MSO', '5G Provider', '5G status', 'img_url', 'launch_type'], dtype='object')

In [55]:
df[pd.isnull(df.launch_type)]['5G Provider'].unique()

array([], dtype=object)

In [56]:
df

,Market,MSO,5G Provider,5G status,img_url,launch_type
0,"Anderson, IN",Comcast,AT&T,Mobile 5G Evolution April 2018,https://static1.squarespace.com/static/58f5270...,Pre 5G / Trial
2,"Atlanta, GA",Comcast/Charter,AT&T,"Launch 5G Mobile Dec. 21, 2018; Mobile 5G Evol...",https://static1.squarespace.com/static/58f5270...,Launch
3,"Aurora, IL",Comcast,AT&T,Mobile 5G Evolution April 2018,https://static1.squarespace.com/static/58f5270...,Pre 5G / Trial
4,"Benton Harbor, MI",Comcast,AT&T,Mobile 5G Evolution October 2018,https://static1.squarespace.com/static/58f5270...,Pre 5G / Trial
5,"Bloomington, IN",Comcast,AT&T,Mobile 5G Evolution April 2018,https://static1.squarespace.com/static/58f5270...,Pre 5G / Trial
6,"Bureau, IL",Comcast,AT&T,Mobile 5G Evolution October 2018,https://static1.squarespace.com/static/58f5270...,Pre 5G / Trial
7,"Charleston, SC",Comcast/Charter,AT&T,Mobile 5G Evolution October 2018,https://static1.squarespace.com/static/58f5270...,Pre 5G / Trial
8,"Chicago, IL",Comcast,AT&T,Mobile 5G Evolution YE 2017,https://static1.squarespace.com/static/58f5270...,Pre 5G / Trial
9,"Citrus, FL",Comcast/Charter,AT&T,Mobile 5G Evolution October 2018,https://static1.squarespace.com/static/58f5270...,Pre 5G / Trial
10,"Fayette, TN",Comcast,AT&T,Mobile 5G Evolution October 2018,https://static1.squarespace.com/static/58f5270...,Pre 5G / Trial


In [57]:
df.to_excel('/drives/shares/OTI Intelligence/5GTRAK/PowerBI reports/5g Market Status/flattened_file.xlsx')

In [58]:
list(df['5G status'].unique())

['Mobile 5G Evolution April 2018',
 'Launch 5G Mobile Dec. 21, 2018; Mobile 5G Evolution YE 2017',
 'Mobile 5G Evolution October 2018',
 'Mobile 5G Evolution YE 2017',
 'Launch 5G Mobile Dec. 21, 2018; Mobile 5G Evolution October 2018',
 'Launch 5G Mobile early 2019; Mobile 5G Evolution YE 2017',
 'Fixed trial with DirecTV Now August 2017; Mobile 5G Evolution October 2018',
 'Mobile 5G Evolution February 2018',
 '5G Mobile Launch early 2019; Mobile 5G Evolution April 2018',
 'Fixed 5G trials Q3-4 2016; fixed trial with DirecTV Now June 2017; fixed/mobile 5G trial at convention center, Mobile 5G Evolution YE 2017',
 'Has filed request for mobile 5Gtest',
 'fixed trial with DirecTV Now August 2017',
 'Launch 5G Mobile 1H 2019',
 'Launch 5G Mobile 1H 2019; Mobile 5G Evolution YE 2017',
 'Launch 5G Mobile 1H 2019; Mobile 5G Evolution April 2018',
 'Launch 5G Mobile Launch 1H 2019; Mobile 5G Evolution YE 2017',
 'Mobile 5G Evolution July 2018',
 'Launch 5G Mobile Dec. 21, 2018; Mobile 5G Ev

In [59]:
f'{2}, {None}:'

'2, None:'

In [60]:
d = {'b': 'c'}

def func1(a='a', b='b'):
    print(f'{a} {b}')
    
def func2(d):
    return func1(**d)

func2(d)

a c


In [61]:
d = 

SyntaxError: invalid syntax (<ipython-input-61-6f2c14ea74e1>, line 1)